In [ ]:
import numpy as np 
import pandas as pd 
import scipy.optimize as opt
import math
from pathlib import Path
from tqdm.notebook import tqdm
import pyproj
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
def ecef2lla(x, y, z):
    # x, y and z are scalars or vectors in meters
    x = np.array([x]).reshape(np.array([x]).shape[-1], 1)
    y = np.array([y]).reshape(np.array([y]).shape[-1], 1)
    z = np.array([z]).reshape(np.array([z]).shape[-1], 1)

    a=6378137
    a_sq=a**2
    e = 8.181919084261345e-2
    e_sq = 6.69437999014e-3

    f = 1/298.257223563
    b = a*(1-f)

    # calculations:
    r = np.sqrt(x**2 + y**2)
    ep_sq  = (a**2-b**2)/b**2
    ee = (a**2-b**2)
    f = (54*b**2)*(z**2)
    g = r**2 + (1 - e_sq)*(z**2) - e_sq*ee*2
    c = (e_sq**2)*f*r**2/(g**3)
    s = (1 + c + np.sqrt(c**2 + 2*c))**(1/3.)
    p = f/(3.*(g**2)*(s + (1./s) + 1)**2)
    q = np.sqrt(1 + 2*p*e_sq**2)
    r_0 = -(p*e_sq*r)/(1+q) + np.sqrt(0.5*(a**2)*(1+(1./q)) - p*(z**2)*(1-e_sq)/(q*(1+q)) - 0.5*p*(r**2))
    u = np.sqrt((r - e_sq*r_0)**2 + z**2)
    v = np.sqrt((r - e_sq*r_0)**2 + (1 - e_sq)*z**2)
    z_0 = (b**2)*z/(a*v)
    h = u*(1 - b**2/(a*v))
    phi = np.arctan((z + ep_sq*z_0)/r)
    lambd = np.arctan2(y, x)

    return phi*180/np.pi, lambd*180/np.pi, h

In [ ]:
def rotate_sat(sat_pos, tm, bias=0):
    periods = 24 * 3600
    res = sat_pos.copy()
    ang = 2 * math.pi / periods * (tm - bias)
    res[:,2] = sat_pos[:,2]
    res[:,0] = np.cos(ang) * sat_pos[:, 0] + np.sin(ang) * sat_pos[:, 1] 
    res[:,1] = - np.sin(ang) * sat_pos[:, 0] + np.cos(ang) * sat_pos[:, 1]
    return res

# def move_sat(sat_pos, sat_vel, pr):
#     print(sat_pos.shape, sat_vel.shape, pr.shape)
#     return sat_pos - sat_vel * pr.reshape(-1, 1) / 299792458

In [ ]:
def calc_pos_fix(sat_pos, pr, weights=1, x0=[0, 0, 0, 0]):
    '''
    Calculates gps fix with WLS optimizer
    returns:
    0 -> list with positions
    1 -> pseudorange errs
    '''
    n = len(pr)
    if n < 3:
        return x0, []
    sat_pos = rotate_sat(sat_pos, pr / 299792458)
    Fx_pos = pr_residual(sat_pos, pr, weights=weights)
    # bounds = (np.array([-np.inf, -np.inf, -np.inf, -100]), np.array([np.inf, np.inf, np.inf, 100]))
    # opt_pos = opt.least_squares(Fx_pos, x0, bounds=bounds).x
    opt_pos = opt.least_squares(Fx_pos, x0).x
    return opt_pos, Fx_pos(opt_pos, weights=1)


def pr_residual(sat_pos, pr, weights=1):
    # solve for pos
    def Fx_pos(x_hat, weights=weights):
        rows = weights * (np.linalg.norm(sat_pos - x_hat[:3], axis=1) + x_hat[3] - pr)
        # rows = weights * (np.linalg.norm(rotate_sat(sat_pos, pr / 299792458, x_hat[3] / 299792458) - x_hat[:3], axis=1) + x_hat[3] - pr)
        return rows
    return Fx_pos

In [ ]:
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
          np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [ ]:
def percentile50(x):
    return np.percentile(x, 50)
def percentile95(x):
    return np.percentile(x, 95)

def get_train_score(df, gt):
    gt = gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
    df = df.merge(gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
    # calc_distance_error
    df['err'] = calc_haversine(df['latDeg_gt'], df['lngDeg_gt'], df['latDeg'], df['lngDeg'])
    # calc_evaluate_score
    df['phone'] = df['collectionName'] + '_' + df['phoneName']
    res = df.groupby('phone')['err'].agg([percentile50, percentile95])
    res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2 
    score = res['p50_p90_mean'].mean()
    return score

In [ ]:
def estimation_pipeline(df_trails):
    """ simple pipeline to estimate the GNSS receiver location by least square
    Args:
    df_trails: the df read from derived file

    Returns:
    result df with estimated degrees and heights
    """
    df_trails["correctedPrM"] = df_trails["rawPrM"] + df_trails["satClkBiasM"] - df_trails["isrbM"] - df_trails["ionoDelayM"] - df_trails["tropoDelayM"]

    results = []
    results_bias = []
    x = [0, 0, 0, 0]
    ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
    
    df_epochs = df_trails.groupby(["collectionName", "phoneName", "millisSinceGpsEpoch"])
    for i, (indices, df_epoch) in enumerate(tqdm(df_epochs, desc="Estimate location by LS for epoch")):
        # if indices[0] == '2020-05-14-US-MTV-1':
        #     if len(results_bias) > 1000:
        #         break
        # else:
        #     continue
        sat_pos = df_epoch[["xSatPosM","ySatPosM","zSatPosM"]].to_numpy()
        pseudoranges = np.squeeze(df_epoch[["correctedPrM"]].to_numpy())
        pseudoranges_sigma = np.squeeze(df_epoch[["rawPrUncM"]].to_numpy())
        x, _ = calc_pos_fix(sat_pos, pseudoranges, 1/pseudoranges_sigma, x)
        # values = np.squeeze(pyproj.transform(ecef, lla, *x[:3], radians=False))
        values = np.squeeze(ecef2lla(*x[:3]))
        results.append([*indices, *values])
        # results_bias.append(x[3])
    return pd.DataFrame(results,columns=["collectionName", "phoneName", "millisSinceGpsEpoch", "latDeg", "lngDeg", "heightAboveWgs84EllipsoidM"])
            # , results_bias

# Estimate whole GNSS


In [ ]:
datapath = Path("../input/google-smartphone-decimeter-challenge/")
ground_truths = (datapath / "train").rglob("ground_truth.csv")
drived_files = (datapath / "train").rglob("*_derived.csv")

df_gt = pd.concat([pd.read_csv(filepath) for filepath in tqdm(ground_truths, total=73, desc="Reading ground truth data")], ignore_index=True)
df_baseline = pd.read_csv(datapath / 'baseline_locations_train.csv')
df_derived = pd.concat([pd.read_csv(filepath) for filepath in tqdm(drived_files, total=73, desc="Reading drived data")], ignore_index=True)

In [ ]:
df_gt["receivedSvTimeInGpsNanos"] = df_gt.millisSinceGpsEpoch*int(1e6)
df_derived_raw = df_derived.drop("millisSinceGpsEpoch", axis=1)

df_merge = pd.merge_asof(df_derived_raw.sort_values('receivedSvTimeInGpsNanos'), df_gt.sort_values('receivedSvTimeInGpsNanos'), 
                                           on="receivedSvTimeInGpsNanos", by=["collectionName", "phoneName"], direction='nearest',tolerance=int(1e9))
df_merge = df_merge.sort_values(by=["collectionName", "phoneName", "millisSinceGpsEpoch"], ignore_index=True)

In [ ]:
# df_estimate = estimation_pipeline(df_merge)
# df_sample_trails_estimate, results_bias = simple_pipeline(df_merge)

In [ ]:
df_merged_baseline = pd.merge_asof(df_gt.sort_values('millisSinceGpsEpoch'),
                                 df_baseline.sort_values('millisSinceGpsEpoch'), 
                                 on="millisSinceGpsEpoch", by=["collectionName", "phoneName"], 
                                 direction='nearest',tolerance=100000, suffixes=('_truth', '_pred'))
df_merged_baseline = df_merged_baseline.sort_values(by=["collectionName", "phoneName", "millisSinceGpsEpoch"], ignore_index=True)

df_merged_SL = pd.merge_asof(df_gt.sort_values('millisSinceGpsEpoch'), 
                           df_estimate.sort_values('millisSinceGpsEpoch'), 
                           on="millisSinceGpsEpoch", by=["collectionName", "phoneName"], 
                           direction='nearest',tolerance=100000, suffixes=('_truth', '_pred'))
df_merged_SL = df_merged_SL.sort_values(by=["collectionName", "phoneName", "millisSinceGpsEpoch"], ignore_index=True)

compared_cols = ["latDeg_truth","lngDeg_truth","latDeg_pred","lngDeg_pred"]
print("Weighted Least Square (baseline) haversine distance (M):", calc_haversine(*df_merged_baseline[compared_cols].to_numpy().transpose()).mean())
print("Weighted Least Square haversine distance (M):", calc_haversine(*df_merged_SL[compared_cols].to_numpy().transpose()).mean())

In [ ]:
df_baseline = df_baseline.drop(["latDeg","lngDeg","heightAboveWgs84EllipsoidM"], axis=1)
df_merged = pd.merge_asof(df_baseline.sort_values('millisSinceGpsEpoch'), 
                            df_estimate.sort_values('millisSinceGpsEpoch'), 
                            on="millisSinceGpsEpoch", by=["collectionName", "phoneName"], direction='nearest', tolerance=100000)
df_merged = df_merged.sort_values(by=["phone", "millisSinceGpsEpoch"], ignore_index=True)

df_submission = df_merged[["phone", "millisSinceGpsEpoch", "latDeg", "lngDeg"]].copy()
df_submission.to_csv('submission.csv', index=False)

In [ ]:
df_baseline = df_baseline.drop(["latDeg","lngDeg","heightAboveWgs84EllipsoidM"], axis=1)
df_merged = pd.merge_asof(df_baseline.sort_values('millisSinceGpsEpoch'), 
                            df_estimate.sort_values('millisSinceGpsEpoch'), 
                            on="millisSinceGpsEpoch", by=["collectionName", "phoneName"], direction='nearest', tolerance=100000)
df_merged = df_merged.sort_values(by=["phone", "millisSinceGpsEpoch"], ignore_index=True)

df_submission = df_merged[["phone", "millisSinceGpsEpoch", "latDeg", "lngDeg"]].copy()
df_submission.to_csv('submission.csv', index=False)

In [ ]:
df_baseline = df_baseline.drop(["latDeg","lngDeg","heightAboveWgs84EllipsoidM"], axis=1)
df_merged = pd.merge_asof(df_baseline.sort_values('millisSinceGpsEpoch'), 
                            df_estimate.sort_values('millisSinceGpsEpoch'), 
                            on="millisSinceGpsEpoch", by=["collectionName", "phoneName"], direction='nearest', tolerance=100000)
df_merged = df_merged.sort_values(by=["phone", "millisSinceGpsEpoch"], ignore_index=True)

df_submission = df_merged[["phone", "millisSinceGpsEpoch", "latDeg", "lngDeg"]].copy()
df_submission.to_csv('submission.csv', index=False)

In [ ]:
df_baseline = df_baseline.drop(["latDeg","lngDeg","heightAboveWgs84EllipsoidM"], axis=1)
df_merged = pd.merge_asof(df_baseline.sort_values('millisSinceGpsEpoch'), 
                            df_estimate.sort_values('millisSinceGpsEpoch'), 
                            on="millisSinceGpsEpoch", by=["collectionName", "phoneName"], direction='nearest', tolerance=100000)
df_merged = df_merged.sort_values(by=["phone", "millisSinceGpsEpoch"], ignore_index=True)

df_submission = df_merged[["phone", "millisSinceGpsEpoch", "latDeg", "lngDeg"]].copy()
df_submission.to_csv('submission.csv', index=False)

In [ ]:
df_baseline = df_baseline.drop(["latDeg","lngDeg","heightAboveWgs84EllipsoidM"], axis=1)
df_merged = pd.merge_asof(df_baseline.sort_values('millisSinceGpsEpoch'), 
                            df_estimate.sort_values('millisSinceGpsEpoch'), 
                            on="millisSinceGpsEpoch", by=["collectionName", "phoneName"], direction='nearest', tolerance=100000)
df_merged = df_merged.sort_values(by=["phone", "millisSinceGpsEpoch"], ignore_index=True)

df_submission = df_merged[["phone", "millisSinceGpsEpoch", "latDeg", "lngDeg"]].copy()
df_submission.to_csv('submission.csv', index=False)

# Test Sanjose

In [ ]:
df_train = pd.read_csv('../input/public-5639-train-test/train_predicted.csv')

datapath = Path("../input/google-smartphone-decimeter-challenge/")
ground_truths = (datapath / "train").rglob("ground_truth.csv")
drived_files = (datapath / "train").rglob("*_derived.csv")

df_gt = pd.concat([pd.read_csv(filepath) for filepath in tqdm(ground_truths, total=73, desc="Reading ground truth data")], ignore_index=True)
df_baseline = pd.read_csv(datapath / 'baseline_locations_train.csv')
df_derived = pd.concat([pd.read_csv(filepath) for filepath in tqdm(drived_files, total=73, desc="Reading drived data")], ignore_index=True)

In [ ]:
df_derived_raw = df_derived.drop("millisSinceGpsEpoch", axis=1)

df_gt["receivedSvTimeInGpsNanos"] = df_gt.millisSinceGpsEpoch*int(1e6)
df_merge_train_gt = pd.merge_asof(df_derived_raw.sort_values('receivedSvTimeInGpsNanos'), df_gt.sort_values('receivedSvTimeInGpsNanos'), 
                                           on="receivedSvTimeInGpsNanos", by=["collectionName", "phoneName"], direction='nearest',tolerance=int(1e9))
df_merge_train_gt = df_merge_train_gt.sort_values(by=["collectionName", "phoneName", "millisSinceGpsEpoch"], ignore_index=True)


df_train["receivedSvTimeInGpsNanos"] = df_train.millisSinceGpsEpoch*int(1e6)
df_merge_train = pd.merge_asof(df_derived_raw.sort_values('receivedSvTimeInGpsNanos'), df_train.sort_values('receivedSvTimeInGpsNanos'), 
                                           on="receivedSvTimeInGpsNanos", by=["collectionName", "phoneName"], direction='nearest',tolerance=int(1e9))
df_merge_train = df_merge_train.sort_values(by=["collectionName", "phoneName", "millisSinceGpsEpoch"], ignore_index=True)

In [ ]:
def get_bias(df_merge_traj):
    ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')

    pos_sat = df_merge_traj[['xSatPosM', 'ySatPosM', 'zSatPosM']].values

    lonlatalt = df_merge_traj[['lngDeg', 'latDeg', 'heightAboveWgs84EllipsoidM']].values
    xyz = pyproj.transform(lla, ecef, *lonlatalt.transpose(), radians=False)
    pos_phone = np.vstack(xyz).transpose()
    
    correctedPrM = df_merge_traj["rawPrM"] + df_merge_traj["satClkBiasM"] - df_merge_traj["isrbM"] - df_merge_traj["ionoDelayM"] - df_merge_traj["tropoDelayM"]
    pr = np.squeeze(correctedPrM.values)

    bias = pr - np.linalg.norm(pos_sat - pos_phone, axis=1)
    return bias

In [ ]:
col = '2021-04-28-US-SJC-1'
# col = '2020-05-14-US-MTV-1'
ph = 'Pixel4'
df_train_traj = df_train[(df_train['collectionName']==col) & (df_train['phoneName']==ph)]
df_gt_traj = df_gt[(df_gt['collectionName']==col) & (df_gt['phoneName']==ph)]

for (collection, phone), df_merge_gt_traj in df_merge_train_gt.groupby(['collectionName', 'phoneName']):
    if collection == col:
        if phone == ph:
            break
        
for (collection, phone), df_merge_train_traj in df_merge_train.groupby(['collectionName', 'phoneName']):
    if collection == col:
        if phone == ph:
            break

In [ ]:
bias_gt = get_bias(df_merge_gt_traj)
bias_train = get_bias(df_merge_train_traj)

df_bias = pd.DataFrame()
df_bias['millisSinceGpsEpoch'] = df_merge_gt_traj['millisSinceGpsEpoch']
df_bias['bias_gt'] = bias_gt
df_bias['bias_train'] = bias_train
df_bias_std = df_bias.groupby('millisSinceGpsEpoch').std()
df_bias_median = df_bias.groupby('millisSinceGpsEpoch').median()

df_bias['bias_gt'].describe()

In [ ]:
dif = np.linalg.norm(df_gt_traj[['latDeg', 'lngDeg']].values - df_train_traj[['latDeg', 'lngDeg']].values, axis=1)

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(311)
ax.scatter(df_merge_gt_traj['millisSinceGpsEpoch'].values, bias_gt, s=1)

ax = fig.add_subplot(312)
ax.scatter(df_merge_gt_traj['millisSinceGpsEpoch'].values, bias_train, s=1)
ax.plot(df_bias_median.index.values,
        # df_bias.groupby('millisSinceGpsEpoch').apply(lambda x: stats.trim_mean(x['bias_train'], 0.3)),
        df_bias_median['bias_train'].rolling(5).mean(),
        label='bias std (train)',
        color='red')
ax.legend()

ax = fig.add_subplot(313)
ax.plot(df_bias_std.index.values, df_bias_std['bias_train'], label='bias std (train)')

ax.plot(df_bias_std.index.values, df_bias_std['bias_gt'], label='bias std (gt)', linestyle='--')
ax.plot(df_gt_traj['millisSinceGpsEpoch'].values, dif * 1e6, label='error')
ax.legend()

# ax = fig.add_subplot(313)
# ax.plot(df_gt_traj['millisSinceGpsEpoch'].values, dif)

In [ ]:
def snap_to_grid_gnss(df_):
    

In [ ]:
def apply_dtw_snap_to_grid(df_input, df_gt, collections_to_snap = None, th_dtw=30.0 / 100_000, th_snap=10.0 / 100_000):
    df_snapped = df_input.copy()
    collections = df_snapped['collectionName'].unique()
    for collection in tqdm(collections, desc = 'Apply DTW snap-to-grid (for {} trajs)'.format("whole" if collections_to_snap is None else len(collections_to_snap))):
        if collections_to_snap is not None:
            if collection not in collections_to_snap:
                continue
        cond_col = df_snapped['collectionName'] == collection
        phones = df_snapped[cond_col]['phoneName'].unique()
        for phone in phones:
            cond_traj = cond_col & (df_snapped['phoneName'] == phone)
            time_traj = df_snapped[cond_traj]['millisSinceGpsEpoch'].values
            latlng_traj = df_snapped[cond_traj][['latDeg', 'lngDeg']].values

            segment_ids_list = get_segment_ids(latlng_traj, base_len=60, stride=30)
            
            snapped_count_list = np.zeros_like(time_traj)
            snapped_results_list = np.zeros((len(time_traj), 2))
            
            for ids in segment_ids_list:
                cond_seg = np.arange(0, latlng_traj.shape[0], 1)
                cond_seg = (cond_seg >= ids[0]) & (cond_seg < ids[1])
                latlng_seg = latlng_traj[cond_seg]
                snapped_count_list += cond_seg
                subtraj_opt, _ = search_closest_subtraj(latlng_seg, df_gt, threshold=th_dtw)
                if subtraj_opt is not None: # もし良い感じのが見つかったら、、、
                    grids = increase_points_array(subtraj_opt)
                    snapped_latlng = snap_to_grid_gnss(latlng_seg, grids, derived, threshold=th_snap)
                    snapped_results_list[ids[0]:ids[1]] += snapped_latlng
                else:
                    snapped_results_list[ids[0]:ids[1]] += latlng_seg
            
            # Add original value if never counted.
            snapped_results_list[snapped_count_list == 0] += latlng_traj[snapped_count_list == 0]
            snapped_count_list[snapped_count_list == 0] += 1

            df_snapped.loc[cond_traj, ['latDeg', 'lngDeg']] = snapped_results_list / snapped_count_list.reshape(-1, 1)
    return df_snapped


def search_closest_subtraj(latlng, df_gt, threshold=30/100_000, expand_idx_len=10):
    
    dist_opt = 1e9
    subtraj_opt = None
    for collection, df_col in df_gt.groupby('collectionName'):
        # Phoneは1つだけ使えばOK
        phone_here = df_col['phoneName'].unique()[0]
        df_traj = df_col[df_col['phoneName'] == phone_here]
        latlng_traj = df_traj[['latDeg', 'lngDeg']].values
        
        dist_start = np.linalg.norm(latlng_traj - latlng[0, :], axis=1)
        dist_end = np.linalg.norm(latlng_traj - latlng[-1, :], axis=1)
        start_cand_idx = np.where(dist_start < threshold)[0]
        end_cand_idx = np.where(dist_end < threshold)[0]

        for start_idx in reduce_array(start_cand_idx):
            for end_idx in reduce_array(end_cand_idx):
                if start_idx >= end_idx:
                    continue
                dist = calc_dtw(latlng[::20], latlng_traj[start_idx: end_idx][::20])[-1][-1][0]
                # try:
                #     dist = calc_dtw(latlng[::20], latlng_traj[start_idx: end_idx][::20])[-1][-1][0]
                # except IndexError as e:
                #     print(start_idx, end_idx)
                if dist_opt > dist:
                    dist_opt = dist
                    subtraj_opt = latlng_traj[max(start_idx-expand_idx_len, 0): end_idx+expand_idx_len]

    return subtraj_opt, dist_opt


def increase_points_array(data, min_dist=0.2/100_000):
    data_increased = []
    for i, point in enumerate(data):
        if i == data.shape[0] - 1:
            continue
        data_increased.append(point)
        if np.array_equal(data[i, 1:], data[i + 1, 1:]):
            continue
        latDeg0 = data[i, 0]
        lngDeg0 = data[i, 1]
        latDeg1 = data[i+1, 0]
        lngDeg1 = data[i+1, 1]
        num_to_increase = np.ceil(np.linalg.norm([latDeg0 - latDeg1, lngDeg0 - lngDeg1]) / min_dist)
        for j in range(int(num_to_increase)):
            latDeg = ((num_to_increase-j) * latDeg0 + j * latDeg1) / num_to_increase
            lngDeg = ((num_to_increase-j) * lngDeg0 + j * lngDeg1) / num_to_increase
            data_increased.append([latDeg, lngDeg])
    data_increased = np.array(data_increased)
    return data_increased


def reduce_array(array, length=50):
    reduced = []
    for idx in array:
        if len(reduced) == 0:
            reduced.append(idx)
        else:
            if reduced[-1] + length <= idx:
                reduced.append(idx)
    return reduced


def get_segment_ids(latlng, base_len=60, stride=30):
    segment_ids = []
    start = 0
    end = start + base_len
    while True:
        if np.linalg.norm(latlng[min(end, latlng.shape[0]-1), :] - latlng[start, :]) > 200/100_000: 
            segment_ids.append([start, end])
            start = end - stride
            end = start + base_len
        else:
            end += stride
        if end >= len(latlng):
            segment_ids.append([latlng.shape[0]-1 - 250, latlng.shape[0]-1])
            break
    return segment_ids

In [ ]:
def snap_to_grid_gnss(latlng_seg, grids, derived, threshold=10/100_000):
    latlng_snapped = copy.copy(latlng_seg)
    for epoch in epochs:
        if min_dist > threshold: # もし距離の近いものがなければそもそも無視。
            continue
        loss_function = get_loss_function(derived_epoch)
        opt_pos = argmin loss_function([grids, bias], weights=weights)
        latlng_snapped[HOGE] = opt_pos
    return latlng_snapped

In [ ]:
df_train_traj